In [2]:
import pandas as pd
import os
import os.path as osp
import numpy as np
import matplotlib.pylab as plt

In [3]:
file_mapping_path = osp.join("..","assets","file_to_session_mapping_20230930-123448.xlsx")
df_map = pd.read_excel(file_mapping_path)

In [4]:
df = df_map.loc[~df_map["video_pose_estimation"].isnull(),['rat_number', 'day_number', 'probe_number',
       'cropped_raw_video', 'video_pose_estimation']]

In [5]:
label_cols = ['rat_number', 'day_number', 'probe_number']

In [6]:
df_sample = df.iloc[1,:]
df_loaded = pd.read_csv(df_sample.video_pose_estimation)


In [7]:
joint_cols = [a for a in df_loaded.columns if '_x' in a or '_y' in a]
prob_cols = [a.replace('_x','_p') for a in joint_cols if '_x' in a]
cols = joint_cols + prob_cols
cols

['Ear_left_1_x',
 'Ear_left_1_y',
 'Ear_right_1_x',
 'Ear_right_1_y',
 'Nose_1_x',
 'Nose_1_y',
 'Center_1_x',
 'Center_1_y',
 'Lat_left_1_x',
 'Lat_left_1_y',
 'Lat_right_1_x',
 'Lat_right_1_y',
 'Tail_base_1_x',
 'Tail_base_1_y',
 'Tail_end_1_x',
 'Tail_end_1_y',
 'Ear_left_2_x',
 'Ear_left_2_y',
 'Ear_right_2_x',
 'Ear_right_2_y',
 'Nose_2_x',
 'Nose_2_y',
 'Center_2_x',
 'Center_2_y',
 'Lat_left_2_x',
 'Lat_left_2_y',
 'Lat_right_2_x',
 'Lat_right_2_y',
 'Tail_base_2_x',
 'Tail_base_2_y',
 'Tail_end_2_x',
 'Tail_end_2_y',
 'Ear_left_1_p',
 'Ear_right_1_p',
 'Nose_1_p',
 'Center_1_p',
 'Lat_left_1_p',
 'Lat_right_1_p',
 'Tail_base_1_p',
 'Tail_end_1_p',
 'Ear_left_2_p',
 'Ear_right_2_p',
 'Nose_2_p',
 'Center_2_p',
 'Lat_left_2_p',
 'Lat_right_2_p',
 'Tail_base_2_p',
 'Tail_end_2_p']

In [8]:
# all_df_list = []
# for index, row in df.iterrows():
#     df_loaded = pd.read_csv(row.video_pose_estimation)
#     df_loaded = df_loaded[cols]
#     for par in label_cols:
#         df_loaded[par] = row[par]
#         all_df_list.append(df_loaded)
    

In [12]:
# all_df_list = [a.reset_index() for a in all_df_list]


In [14]:
# all_df = pd.concat(all_df_list)

In [ ]:
# all_df.rename(columns={"index":"frame"}, inplace=True)

In [15]:
# all_df.set_index(label_cols, inplace=True)
# all_df = pd.read_csv("combined_pose_estimation.csv")

In [16]:
# all_df.to_csv("combined_pose_estimation.csv")

In [17]:
unique_inds = all_df.index.unique()
one_rec = all_df.loc[unique_inds[0]]

In [18]:
one_rec[joint_cols]

Ear_left_1_x     431.532736
Ear_left_1_y     124.633394
Ear_right_1_x    431.133034
Ear_right_1_y     80.178687
Nose_1_x         403.816611
Nose_1_y         107.888238
Center_1_x       444.671415
Center_1_y       101.585715
Lat_left_1_x     494.763195
Lat_left_1_y      78.676210
Lat_right_1_x    481.424115
Lat_right_1_y     76.905941
Tail_base_1_x    494.457150
Tail_base_1_y    134.171430
Tail_end_1_x     502.817444
Tail_end_1_y      93.740761
Ear_left_2_x     148.546154
Ear_left_2_y     111.150253
Ear_right_2_x    184.984047
Ear_right_2_y    113.048882
Nose_2_x         176.792152
Nose_2_y          77.346106
Center_2_x       167.900000
Center_2_y       125.728570
Lat_left_2_x     231.344913
Lat_left_2_y     207.364955
Lat_right_2_x    224.762748
Lat_right_2_y    214.347996
Tail_base_2_x    174.585710
Tail_base_2_y    236.600000
Tail_end_2_x     196.610829
Tail_end_2_y     225.063948
Name: 0, dtype: float64

In [ ]:
fps = 1/30
n_frames_per_sec = 1/fps
n_sec_per_seg = 2
n_frames_per_seg = np.floor(n_sec_per_seg * n_frames_per_sec)
n_frames_per_seg
overlap_percent = 0.5

In [ ]:
n_frames_per_seg

In [ ]:
import numpy as np
import pywt

def sliding_cwt(data, num_frames_per_segment, overlap_percent):
    if overlap_percent < 0 or overlap_percent >= 1:
        raise ValueError("Overlap percent must be between 0 and 1")
        
    bandwidth_frequency = 1.5
    center_frequency = 1
    cmor = f'cmor{bandwidth_frequency}-{center_frequency}'
    
    step_size = int(np.ceil(num_frames_per_segment * overlap_percent))
    num_steps = int(np.ceil((len(data) - num_frames_per_segment) / step_size)) + 1
    num_frames_per_segment = int(num_frames_per_segment)

    
    cwt_matrix = np.empty((num_steps, *pywt.cwt(data[:num_frames_per_segment], [1], cmor, num_frames_per_segment).shape))
    
    for i in range(num_steps):
        start_idx = i * step_size
        end_idx = start_idx + num_frames_per_segment
        segment = data[start_idx:end_idx]
        cwt = pywt.cwt(segment, [1], cmor, num_frames_per_segment)
        cwt_matrix[i] = cwt

    return cwt_matrix

In [ ]:
data = one_rec[joint_cols[0]].values


In [ ]:
data[:10]

In [ ]:
current_dir = os.getcwd()
os.chdir("..")
from utils.helper_functions import continuous_wavelet_transform
os.chdir(current_dir)

In [ ]:
bandwidth_frequency = 1.5
center_frequency = 1
cmor = f'cmor{bandwidth_frequency}-{center_frequency}'
num_frames_per_segment = n_frames_per_seg

step_size = int(np.ceil(num_frames_per_segment * overlap_percent))
num_steps = int(np.ceil((len(data) - num_frames_per_segment) / step_size)) + 1
num_frames_per_segment = int(num_frames_per_segment)
 
    
f_min = 1  # Minimum frequency
f_max = 30  # Maximum frequency

# Define the number of frequencies you want (e.g., 10)
num_frequencies = 10

# Calculate the scales for the desired frequencies
# scales = [1 / (2 * np.pi * freq) for freq in np.linspace(f_min, f_max, num_frequencies)]

scales = np.linspace(f_min, f_max, num_frequencies)
    
cwt_result, freqs = pywt.cwt(data[:num_frames_per_segment], scales, cmor, sampling_period=2)


In [ ]:
cwt_result

In [ ]:
f0 = 1
fn = 10
num_frequencies = 50
fs = 30
wavelet = 'morl'
num_frames_per_segment = int(fs*30000)
frequencies = np.linspace(f0, fn, num_frequencies)
scale = pywt.frequency2scale(cmor, frequencies)
cwt_result, freqs = pywt.cwt(data, scale, wavelet)

t = np.arange(0, cwt_result.shape[1]) / fs
# cwt_result = np.abs(cwt_result)
cwt_result /= np.abs(cwt_result).max()


In [ ]:
t_0 = 20
step_size = 10

plot_ind = range(int(fs*t_0), int(fs*(t_0+step_size)))
plt.pcolormesh(t[plot_ind], frequencies ,cwt_result[:,plot_ind], shading='gouraud')

In [ ]:
data.shape[0]

In [ ]:
np.linspace(t_0, data.shape[0]/fs-plot_range, 10)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Assuming you have already defined t, frequencies, fs, and cwt_result
fs = 29
t_0 = 0
plot_range = 20
plot_ind = range(int(fs * t_0), int(fs * (t_0 + plot_range)))

fig, ax = plt.subplots()
pcm = ax.pcolormesh(t[plot_ind], frequencies, cwt_result[:, plot_ind], shading='gouraud')
ax.set_xlabel('Time')
ax.set_ylabel('Frequencies')
ax.set_title(f't_0 = {t_0}')

def update(t_0):
    plot_ind = range(int(fs * t_0), int(fs * (t_0 + plot_range)))
    pcm.set_array(cwt_result[:, plot_ind])
    ax.set_title(f't_0 = {t_0}')
    return pcm,

# Define the range of t_0 values for the animation
t_0_values = np.linspace(t_0, data.shape[0]/fs-plot_range, 100)

ani = FuncAnimation(fig, update, frames=t_0_values, blit=True)
# ani.save('cwt_animation.gif', writer='pillow')  # You can save the animation as a GIF

# In Jupyter Notebook, you can display the animation using:
from IPython.display import HTML
HTML(ani.to_jshtml())


In [ ]:
time, frequencies, cwt_matrix = continuous_wavelet_transform(
    data[:num_frames_per_segment], fs=30, f0=1e-5, fn=10, num_frequencies=10, wavelet='morl')
cwt_matrix = np.abs(cwt_matrix)
plt.pcolormesh(time, frequencies, cwt_matrix)

In [ ]:
cwt_out = np.abs(cwt_result)
plt.pcolormesh(cwt_out, shading='gouraud')

In [ ]:
# detemine the length of the videos
from moviepy.editor import VideoFileClip
all_durations = 0
video_paths = df['cropped_raw_video'].values
with VideoFileClip(video_path) as video:
    duration = video.duration
    all_durations += duration

In [1]:
import moviepy

ModuleNotFoundError: No module named 'moviepy'